In [1]:
import tensorflow as tf
from tensorflow.python.tools import freeze_graph, optimize_for_inference_lib
tf.__version__

'1.12.0'

In [6]:
MODEL_NAME = 'face_model'

# Freeze the graph

input_graph_path = './tmp/'+MODEL_NAME+'.pbtxt'

with tf.Session() as sess:
    
    
    saver = tf.train.import_meta_graph('./tmp/model.ckpt.meta')
    saver.restore(sess, './tmp/model.ckpt')
    
    checkpoint_path = './tmp/model.ckpt'
    input_saver_def_path = ""
    input_binary = False
    output_node_names = "layer22/leaky_relu22_output"
    restore_op_name = "save/restore_all"
    filename_tensor_name = "save/Const:0"
    output_frozen_graph_name = './tmp/frozen_'+MODEL_NAME+'.pb'
    
    clear_devices = True
    

    freeze_graph.freeze_graph(input_graph_path, input_saver_def_path,
                              input_binary, checkpoint_path, output_node_names,
                              restore_op_name, filename_tensor_name,
                              output_frozen_graph_name, clear_devices, "")

        


INFO:tensorflow:Restoring parameters from ./tmp/model.ckpt
INFO:tensorflow:Restoring parameters from ./tmp/model.ckpt
INFO:tensorflow:Froze 111 variables.
INFO:tensorflow:Converted 111 variables to const ops.


In [12]:

output_optimized_graph_name = 'optimized_'+MODEL_NAME+'.pb'

input_graph_def = tf.GraphDef()
with tf.gfile.Open(output_frozen_graph_name, "rb") as f:
    data = f.read()
    input_graph_def.ParseFromString(data)


    
#tf.graph_util.remove_training_nodes(
#    input_graph_def,
#    protected_nodes=None
#)
for node in input_graph_def.node:

    if node.op == "Switch":
        node.op = "Conv2D"
    #if node.op == "Placeholder":
        #print(node)
        #node.op = "Const"
        #d.del(node.op)
        #tf.google.c
        #node.op = "Const"
        #print(type(node.attr['value'].type))
        #print(node.attr["value"].tensor)
        #del node.attr['value']
        #node.attr['key', ['value'].type] = ['dtype', 1 ]
        #print(node)
        #print(node)
    #if node.attr["value"].type == 0:
        #print(node)
         
    
output_graph_def = optimize_for_inference_lib.optimize_for_inference(
    input_graph_def,
    ["input"], # an array of the input node(s)
    ["layer22/leaky_relu22_output"], # an array of output nodes
    tf.float32.as_datatype_enum)



# Save the optimized graph




name: "sub/y"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
      }
      float_val: 0.0
    }
  }
}

name: "layer1/weights"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
        dim {
          size: 3
        }
        dim {
          size: 3
        }
        dim {
          size: 3
        }
        dim {
          size: 32
        }
      }
      tensor_content: "[\321\006<I\300\001:\310\373o9\201\244\333;X-\233\272{\014\2778-\001\230;\325\334\275;\202w\212\271y\3524\272\312\021\303\272\274x\223\272\036`\003<\372\275\014:\000v\021\274E\001m\273\002\3111<nX\360\2737\027\266\273\243\225Q\2744\346 \273\372%S\274v\324\030;\206>!\272afB;\207\000\216\273\002\370\n\274,r\233;\357\260\002<7?\004<\375\033\312\273\226\210\030<f\226\266;\025\230\241\274\034\242\243;\364\367

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
f=tf.gfile.FastGFile('./tmp/'+output_optimized_graph_name, "w")
f.write(output_graph_def.SerializeToString())


In [29]:
with tf.gfile.FastGFile('./tmp/'+output_optimized_graph_name, "w") as f:
    f.write(output_graph_def.SerializeToString())


In [9]:
with tf.python_io.TFRecordWriter('./tmp/'+output_optimized_graph_name) as writer:
     writer.write(output_graph_def.SerializeToString())

In [14]:
graph_def_file = './tmp/optimized_'+MODEL_NAME+'.pb'
input_arrays = ["input"]
output_arrays = ["layer22/layer22_relu_output"]

converter = tf.contrib.lite.TFLiteConverter.from_frozen_graph(
  graph_def_file, input_arrays, output_arrays)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

ValueError: Invalid tensors 'last_tensor' were found.